In [1]:
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train = pd.read_csv('processed_data/train_translated_cleaned.csv')
test = pd.read_csv('processed_data/test_translated_cleaned.csv')
test_y = pd.read_csv("processed_data/test_labels_cleaned.csv")

In [3]:
train['comment_text'] = train['comment_text'].apply(lambda x: np.str_(x))
test['comment_text'] = test['comment_text'].apply(lambda x: np.str_(x))

#split 20% of train as validation set
X = train.iloc[:,1]
y = train.iloc[:,2:]

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=321)

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63978 entries, 0 to 63977
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            63978 non-null  object
 1   comment_text  63978 non-null  object
dtypes: object(2)
memory usage: 999.8+ KB


In [5]:
#convert text to TFIDF
tf_vec = TfidfVectorizer(decode_error='ignore',stop_words='english')
train_tf = tf_vec.fit_transform(X_train)
validation_tf = tf_vec.transform(X_validation)

In [6]:
len(tf_vec.get_feature_names())

136829

In [7]:
#train different model for each toxicity
col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
# create empty array to contain predictions
pred_validation = np.zeros((X_validation.shape[0],len(col)))
nb = MultinomialNB()
for i,x in enumerate(col):
    print('####Working on ',x , ' ####')
    nb.fit(train_tf, y_train[x])
    pred_validation[:,i] = nb.predict(validation_tf)

####Working on  toxic  ####
####Working on  severe_toxic  ####
####Working on  obscene  ####
####Working on  threat  ####
####Working on  insult  ####
####Working on  identity_hate  ####


In [8]:
for i,x in enumerate(col):
    print(x,"Accuracy Score on validation set:",metrics.accuracy_score(y_validation[x], pred_validation[:,i]))

toxic Accuracy Score on validation set: 0.9243929186902711
severe_toxic Accuracy Score on validation set: 0.9897227009243302
obscene Accuracy Score on validation set: 0.9552561491461695
threat Accuracy Score on validation set: 0.9971173429421902
insult Accuracy Score on validation set: 0.9526241579194736
identity_hate Accuracy Score on validation set: 0.9906313645621181


In [9]:
#Tuning parameter alpha
alpha = [0.01, 0.015, 0.1, 0.5, 1]

best_accuracy = 0
best_alpha = 0

col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
# create empty array to contain predictions
pred_validation = np.zeros((X_validation.shape[0],len(col)))

for i,x in enumerate(col):
    for j in alpha:
        nb = MultinomialNB(alpha = j)
        nb.fit(train_tf, y_train[x])
        pred_validation[:,i] = nb.predict(validation_tf)
        temp = metrics.accuracy_score(y_validation[x], pred_validation[:,i])
        if temp > best_accuracy:
            best_accuracy = temp
            best_alpha = j
    print('The best alpha value is ', best_alpha, ' for ', x, '; The accuracy score is: ', best_accuracy)
    best_alpha = 0
    best_accuracy = 0

The best alpha value is  0.015  for  toxic ; The accuracy score is:  0.9472035093216356
The best alpha value is  0.1  for  severe_toxic ; The accuracy score is:  0.9898167006109979
The best alpha value is  0.015  for  obscene ; The accuracy score is:  0.9707660974463418
The best alpha value is  0.015  for  threat ; The accuracy score is:  0.9971800093999686
The best alpha value is  0.015  for  insult ; The accuracy score is:  0.9654081153062823
The best alpha value is  0.015  for  identity_hate ; The accuracy score is:  0.9911953626821244


From above results we can see the best alpha is 0.1 for toxic and obscene, 0.5 for severe_toxic, 0.01 for threat, 0.015 for insult and identity_hate.

In [10]:
pred_test = np.zeros((test.shape[0],len(col)))
col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
#create a array contain best parameter alpha
best_alpha = [0.015, 0.1, 0.015, 0.015, 0.015, 0.015]

#train_tf = tf_vec.fit_transform(train['comment_text'])
test_tf = tf_vec.transform(test['comment_text'])


#log start time
starttime = datetime.datetime.now()

for i,x in enumerate(col):
    print('####Working on ',x , ' ####')
    nb = MultinomialNB(alpha = best_alpha[i])
    nb.fit(train_tf, y_train[x])
    pred_test[:,i] = nb.predict(test_tf)

#log end time
endtime = datetime.datetime.now()

####Working on  toxic  ####
####Working on  severe_toxic  ####
####Working on  obscene  ####
####Working on  threat  ####
####Working on  insult  ####
####Working on  identity_hate  ####


In [11]:
#results
for i,x in enumerate(col):
    print(x,"Accuracy Score on Test:",metrics.accuracy_score(test_y[x], pred_test[:,i]))
print('Total training time: ' , endtime - starttime)

toxic Accuracy Score on Test: 0.9302572759386039
severe_toxic Accuracy Score on Test: 0.99306011441433
obscene Accuracy Score on Test: 0.9595642252024134
threat Accuracy Score on Test: 0.9959361030354185
insult Accuracy Score on Test: 0.9571102566507237
identity_hate Accuracy Score on Test: 0.9889024352121042
Total training time:  0:00:00.518223


In [12]:
total_score = 0
for i,x in enumerate(col):
    score = metrics.accuracy_score(test_y[x], pred_test[:,i])
    total_score= total_score + score
print('The average score for six toxicities is: ', round(total_score/6 , 4) )

The average score for six toxicities is:  0.9708
